In [1]:
from google.colab import drive
import os
import pandas as pd

In [4]:
current_directory = "/data"
os.chdir(current_directory)

In [5]:
!pip install ntlk
!pip install num2words
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement ntlk (from versions: none)
ERROR: No matching distribution found for ntlk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=99bce473c0f841f10e6f70b05dd12b6f02d67ca929de59147018d5a6706d5ad6
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 K

In [6]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from nltk import word_tokenize, sent_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree

import re
import num2words
import json

In [7]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [8]:
!pip install transformers
!pip install sentencepiece
!pip install deepmultilingualpunctuation
from deepmultilingualpunctuation import PunctuationModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
#For segmentation of transcript into product reviews
!pip install ruptures
!pip install sentence_transformers
import ruptures as rpt
from ruptures.base import BaseCost
from sentence_transformers import SentenceTransformer, util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=21f6c528cedd06ecead94eaca7ae9179a938dce6deff6217a21049fd7172a3c0
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


Intialise Classes for Product Review Identification and Segmentation

In [10]:
import math

class ProductIdentifier:
  def __init__(self):
    pass

  def remove_non_ascii(self, text):
    return re.sub(r'[^\x00-\x7F]+',' ', text)

  def GetNounsWithNLTK(self, text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
      if type(subtree) == Tree:
        current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
      elif current_chunk:
        named_entity = " ".join(current_chunk)
        if named_entity not in continuous_chunk:
          continuous_chunk.append(named_entity)
          current_chunk = []
      else:
        continue

    return continuous_chunk

  def get_uppercased_word_series(self, text):
    temp_lst = []
    uppercased_word_lst = []
    word_lst = nltk.word_tokenize(text)
    for i in range(len(word_lst)):
      word = word_lst[i]
      starts_with_uppercase = word[0].isupper()
      if starts_with_uppercase:
        temp_lst.append(word)
        if (i == len(word_lst)-1):
          if len(temp_lst) >= 2:
            uppercased_word_lst.append(" ".join(temp_lst))
        continue
      else:
        if len(temp_lst) >= 2:
          uppercased_word_lst.append(" ".join(temp_lst))
        temp_lst = []
    return list(set(uppercased_word_lst))

  def Check_If_NP_Is_Known_Product(self,np, product_lexicon):
    for product in product_lexicon:
      if np == product:
        return True
    return False

  def Check_If_NP_Has_Product_Indicator(self, np, product_indicators):
    for product in product_indicators:
      similarity = fuzz.partial_ratio(np.lower(),product.lower())
      if similarity >= 80:
        return True
    return False

  def Check_If_NP_Is_Product(self, np, product_indicators, product_lexicon):
    if self.Check_If_NP_Is_Known_Product(np, product_lexicon):
      return True
    elif self.Check_If_NP_Has_Product_Indicator(np, product_indicators):
      return True
    else:
      return False

  def is_not_substring_of_another_product(self,product, product_lst):
    for comparison_product in product_lst:
      if product.lower() == comparison_product.lower():
        continue
      if product.lower() in comparison_product.lower():
        return False
    return True

  def detect_products_from_transcript(self, text, product_indicators, product_lexicon):
    #sometimes nltk ne chunker misses out
    nouns_in_transcript = self.GetNounsWithNLTK(text)
    #parse for a series of uppercase
    uppercased_series_lst = self.get_uppercased_word_series(text)

    #nouns_in_transcript.extend(np_lst)
    nouns_in_transcript.extend(uppercased_series_lst)

    #only keep if len > 2 
    nouns_in_transcript = list(filter(lambda noun_phrase: len(nltk.word_tokenize(noun_phrase))>=2, nouns_in_transcript))

    #keep if match with brand name or contain a product term
    nouns_in_transcript = list(filter(lambda noun_phrase: self.Check_If_NP_Is_Product(noun_phrase, product_indicators, product_lexicon), nouns_in_transcript))

    #filter the products
    nouns_in_transcript = list(filter(lambda product: self.is_not_substring_of_another_product(product, nouns_in_transcript), nouns_in_transcript))

    return list(set(nouns_in_transcript))


  def contains_url(self,text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex, text)
    if url != []:
      return True
    else:
      return False

  def remove_url(self,text):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    removed_url_text = re.sub(regex, "", text)
    return removed_url_text

  def clean_desc(self,text):
    text = text.replace(":", "")
    text = text.replace("*", "")
    text = self.remove_non_ascii(text)
    text = self.remove_url(text)
    text = re.sub(r'#\S+', '', text)
    text = text.replace("https","")
    return text.strip()

  
  def GetNPWithSyntaxRules(self, text):
    nounphrases = []
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)
    chunk_name_lst = ["NP","Product"]
    grammar = r"""  NP:
              {<NN|NNS|NNP|NNPS>+}
              Product:
              {<NN.*|NNS.*|NNP.*|NNPS.*><NN|VBZ|JJ|JJS|JJR|NNS|NNP|NNPS|VBD|CD|RB>+}
          """
    chunkParser=nltk.RegexpParser(grammar)
    tree = chunkParser.parse(tagged)
    for subtree in tree.subtrees(filter=lambda t: t.label() in chunk_name_lst):
        myPhrase = ''
        for item in subtree.leaves():
            myPhrase += ' ' + item[0]
        nounphrases.append(myPhrase.strip())
    nounphrases = list(filter(lambda x: len(x.split()) > 1, nounphrases))
    return nounphrases

  def get_products_from_desc(self,description):
    if description == None:
      return []
    split_desc = description.split("\n")
    #set higher threshold because even if e.g. "mal" "ibu" match seperately -- it raises the match score
    products = []
    for sent in split_desc:
      if sent != "":
        products.extend(self.GetNPWithSyntaxRules(sent))
    products = [self.clean_desc(txt) for txt in products]
    products = list(filter(lambda txt: self.Check_If_NP_Is_Product(txt, product_indicators, product_lexicon),products))
    products = list(filter(lambda txt: self.is_not_substring_of_another_product(txt,products),products))
    return products
  
  def detect_products(self,text,description,product_indicators, product_lexicon):
    #filtered_transcript_product_lst = self.detect_products_from_transcript(text, product_indicators, product_lexicon)
    filtered_transcript_product_lst = self.get_products_from_desc(description)
    #filtered_transcript_product_lst.extend(filtered_desc_product_lst)
    filtered_product_lst = list(filter(lambda txt: self.is_not_substring_of_another_product(txt,filtered_transcript_product_lst),filtered_transcript_product_lst))
    filtered_product_lst = list(set(filtered_product_lst))
    return filtered_product_lst

class CoreferenceResolver:
  def __init__(self):
    self.product_identifier = ProductIdentifier()
  
  def GetCoreferenceWithSyntaxRules(self,text):
    coref = []
    words = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(words)
    chunk_name_lst = ["DT","DT Ref"]
    grammar = r"""
              DT Ref:
              {<DT><CD>}
          """
    chunkParser=nltk.RegexpParser(grammar)
    tree = chunkParser.parse(tagged)
    for subtree in tree.subtrees(filter=lambda t: t.label() in chunk_name_lst):
      myPhrase = ''
      for item in subtree.leaves():
        myPhrase += ' ' + item[0]
      coref.append(myPhrase.strip())
    coref = list(filter(lambda x: len(x.split()) > 1, coref))
    filtered_coref = list(set(coref))
    return filtered_coref

  def replace_all(self, text, dic):
    for i, j in dic.items():
        text = re.sub(r"\b%s\b" % i, j, text)
        # r"\b%s\b"% enables replacing by whole word matches only
    return text

  def detect_matching_product_name(self, sentence, product_lst,product_indicators, product_lexicon):
    np_lst = self.product_identifier.GetNPWithSyntaxRules(sentence)
    for np in np_lst:
      for product in product_lst:
        similarity_with_identified_products = fuzz.partial_ratio(np.lower(),product.lower())
        if similarity_with_identified_products >= 80 or np in product:
          return (np,product)
    for np in np_lst:
      if self.product_identifier.Check_If_NP_Is_Product(np, product_indicators, product_lexicon):
        return (np,np)
    return None


  def resolve(self, sample_sentences, product_lst, product_indicators, product_lexicon):
    curr_product_name = None
    resolved_combined_sentence_list = []
    for sentence in sample_sentences:
      #update current product name
      potentially_new_product_name_pair = self.detect_matching_product_name(sentence, product_lst,product_indicators, product_lexicon)
      if potentially_new_product_name_pair != None and potentially_new_product_name_pair != curr_product_name:
        #must identify the product name better
        phrase = potentially_new_product_name_pair[0]
        curr_product_name = potentially_new_product_name_pair[1]
        #pos_tag sentence --> assign determiners after the product name to the product name
        substr_before_product = sentence.split(phrase,1)[0] + curr_product_name
        substr_after_product = sentence.split(phrase,1)[-1]
        replacements = {'This':curr_product_name,'this':curr_product_name,'it':curr_product_name, 'It':curr_product_name}
        substr_after_product = self.replace_all(substr_after_product, replacements)
        resolved_combined_sentence_list.append(substr_before_product + substr_after_product)
      
      elif curr_product_name != None:
        #replace all references with the curr_product_name
        replacements = {'This':curr_product_name,'this':curr_product_name,'it':curr_product_name, 'It':curr_product_name}
        sentence = self.replace_all(sentence, replacements)
        resolved_combined_sentence_list.append(sentence)
      
      else:
        resolved_combined_sentence_list.append(sentence)
    
    return resolved_combined_sentence_list

  def resolve_sentences(self,sample_sentences, product_lst,product_indicators, product_lexicon):
    sample_sentences = [sentence.replace("\n", " ") for sentence in sample_sentences]
    resolved_list = self.resolve(sample_sentences, product_lst,product_indicators, product_lexicon)
    return resolved_list

#Define cost function for segmentation
class CosineCost(BaseCost):
    """Cost derived from the cosine similarity."""

    # The 2 following attributes must be specified for compatibility.
    model = "custom_cosine"
    min_size = 2

    def fit(self, signal):
        """Set the internal parameter."""
        self.signal = signal
        self.gram = util.cos_sim(signal, signal)
        return self

    def error(self, start, end) -> float:
        """Return the approximation cost on the segment [start:end].

        Args:
            start (int): start of the segment
            end (int): end of the segment
        Returns:
            segment cost
        Raises:
            NotEnoughPoints: when the segment is too short (less than `min_size` samples).
        """
        if end - start < self.min_size:
            raise NotEnoughPoints
        sub_gram = self.gram[start:end, start:end]
        val = sub_gram.diagonal().sum()
        val -= sub_gram.sum() / (end - start)
        return val

class TranscriptSegmenter:
  def __init__(self):
    self.product_identifier = ProductIdentifier()
    self.coreference_resolver = CoreferenceResolver()
    self.punctuator = PunctuationModel()
    self.segmentation_cost_function_class = CosineCost()
    self.sentence_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

  def further_split_sentence(self, nltk_sent_tokenized_list):
    #use punctuator to improve splitting
    final_sent_lst = []
    for sent in nltk_sent_tokenized_list:
      punctuated_sent = self.punctuator.restore_punctuation(sent)
      split_sent = punctuated_sent.split(".")
      final_sent_lst.extend(split_sent)
    return final_sent_lst

  def get_sentence_list_from_transcript(self, transcript):
    transcript = self.product_identifier.remove_non_ascii(transcript)
    phrases = self.further_split_sentence(nltk.sent_tokenize(transcript))
    if len(phrases) == 0:
      return []
    phrases = list(filter(lambda phrase: phrase != "", phrases))
    pos_tagged = [pos_tag(nltk.word_tokenize(sentence.lower())) for sentence in phrases]
    new_combined_sentence_list = [""]
    for i in range(len(phrases)):
      tagged_phrase = pos_tagged[i]
      tag_of_first_word = tagged_phrase[0][1]
      conjunctions_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "RB"]
      end_lst = ["IN", "CC", "WDT", "TO", "WRB", "VBG", "VBZ", "DT", "RB"]
      join_sentence = False
      if i >= 1:
        last_tag_in_prev_sentence = pos_tagged[i-1][-1][1]
        if last_tag_in_prev_sentence in end_lst:
          join_sentence = True
      if tag_of_first_word in conjunctions_lst:
        join_sentence = True

      if join_sentence:
        #lowercase the first letter to reduce noise when doing NP identification e.g. "Which is so beautiful" --> Which to which
        sub_clause = phrases[i].replace(phrases[i][0],phrases[i][0].lower(),1)
        new_combined_sentence_list[-1] = new_combined_sentence_list[-1] + " " + sub_clause
      else:
        new_combined_sentence_list.append(phrases[i])
    return new_combined_sentence_list

  def segment_transcript_ruptures(self, filtered_product_lst, resolved_sentences, original_sentences, product_indicators, product_lexicon):
    #Find breaks where similarity between chunks is the lowest
    num_products = len(filtered_product_lst)
    embeddings = self.sentence_embedding_model.encode(resolved_sentences, convert_to_tensor=True)
    n_bkps = num_products-1+2  # e.g. there are 9 change points (10 text segments) + intro and outrp
    min_size = 2
    jump = 1
    #to create valid segmentation paramters
    if n_bkps > len(embeddings) or n_bkps * math.ceil(min_size/ jump) * jump + min_size > len(embeddings): 
      n_bkps = round(math.ceil(min_size/ jump) * jump + min_size)
    algo = rpt.Dynp(custom_cost=self.segmentation_cost_function_class, min_size=min_size, jump=jump).fit(embeddings)
    predicted_bkps = algo.predict(n_bkps=n_bkps)
    iteration_lst = predicted_bkps
    iteration_lst.insert(0, 0)

    chunk_lst = []
    product_review_dict = {}
    for i in range(len(iteration_lst)-1):
      chunk_resolved = " ".join(resolved_sentences[iteration_lst[i]:iteration_lst[i+1]])
      chunk_org = " ".join(original_sentences[iteration_lst[i]:iteration_lst[i+1]])
      product = self.coreference_resolver.detect_matching_product_name(chunk_resolved, filtered_product_lst,product_indicators, product_lexicon)
      if product != None:
        product_review_dict[product[1]] = {"original":chunk_org}
    return product_review_dict

  def segment_transcript(self, filtered_product_lst, resolved_list, new_combined_sentence_list):
    product_review_dict = {}
    for i in range(len(resolved_list)):
      resolved_sentence = resolved_list[i]
      for product in filtered_product_lst:
        if product in resolved_sentence:
          if product in product_review_dict:
            #product_review_dict[product]["resolved"] = product_review_dict[product]["resolved"] + resolved_sentence
            product_review_dict[product]["original"] = product_review_dict[product]["original"] + new_combined_sentence_list[i]
          else:
            product_review_dict[product] = {"original":new_combined_sentence_list[i]}
    
    return product_review_dict

  def split_transcript_by_detected_products(self, review_dict, product_indicators, product_lexicon):
    transcript = review_dict["transcript_text"]
    description = review_dict["description"]
    filtered_product_lst = self.product_identifier.detect_products(transcript, description, product_indicators, product_lexicon)
    sample_sentences = self.get_sentence_list_from_transcript(transcript)
    resolved_lst = self.coreference_resolver.resolve_sentences(sample_sentences, filtered_product_lst,product_indicators, product_lexicon)
    #product_review_dict = self.segment_transcript(filtered_product_lst, resolved_lst, sample_sentences)
    product_review_dict = self.segment_transcript_ruptures(filtered_product_lst, resolved_lst, sample_sentences, product_indicators, product_lexicon)
    return product_review_dict

  #Case of video having chapters

  def process_raw_transcript(self, raw_transcript):
    text_list = [segment["text"] for segment in raw_transcript]
    text = " ".join(text_list)
    text = re.sub("\n", " ", text)
    #remove non-ascii characters -- to remove foreign lang characters
    text = self.product_identifier.remove_non_ascii(text)
    return text

  def split_transcript_by_chapters(self, raw_transcript, chapters):
    starts = [round(segment["start"]) for segment in raw_transcript]
    product_review_dict = {}
    chapter_index_list = []
    for chapter in chapters:
      time_in_sec = round(chapter["time_in_min"] * 60)
      #get a index list of chapters 
      new_insertion_index = bisect.bisect_left(starts, time_in_sec)
      chapter_index_list.append(new_insertion_index)
    chapter_index_list.append(len(raw_transcript)-1)
    for i in range(len(chapter_index_list)-1):
      title = chapters[i]["title"]
      start_index = chapter_index_list[i]
      end_index = chapter_index_list[i+1]
      original_raw_transcript = self.process_raw_transcript(raw_transcript[start_index:end_index])
      sample_sentences = [segment["text"] for segment in raw_transcript[start_index:end_index]]
      #resolved_raw_transcript = self.coreference_resolver.resolve_sentences(sample_sentences, [title])
      product_review_dict[title] = {"original":original_raw_transcript}
    return product_review_dict


  def get_product_review_dict(self, review_dict, product_indicator, product_lexicon):
    chapters = sample_review_dict["chapters"]
    product_review_dict = {}
    if chapters == []:
      product_review_dict = self.split_transcript_by_detected_products(review_dict, product_indicator, product_lexicon)
    else:
      raw_transcript = sample_review_dict["transcript_raw"]
      product_review_dict = self.split_transcript_by_chapters(raw_transcript, chapters)
    return product_review_dict


Load Youtube Data

In [11]:
import bisect
import json
from fuzzywuzzy import fuzz

In [ ]:
def get_beauty_lexicon(path):
    beauty_lexicon = []
    df = pd.read_csv(path)
    beauty_lexicon = df["Lexicon"].unique().tolist()
    return beauty_lexicon

product_indicators = get_beauty_lexicon("beauty_lexicon.csv")
product_lexicon = pd.read_csv("Product Lexicon.csv")["Product"].to_list()

#will take longer on initial load due to loading of punctuator model
transcript_segmenter = TranscriptSegmenter()

Demo Example (On video without chapters)

In [14]:


#demo on yjYaxtgVfyg from Feb23 data
input_json_filename = "Feb23.json"
output_json_filename = "PRI_Output_"+input_json_filename
#Load data scraped using various Youtube API
with open(input_json_filename, 'r') as f:
  reviews = json.load(f)

sample_review_dict = reviews["yjYaxtgVfyg"]
product_review_dict = transcript_segmenter.get_product_review_dict(sample_review_dict, product_indicators, product_lexicon)
product_review_dict

print("Without chapters")
for k, v in product_review_dict.items():
  print(k)
  print(v)

sample_review_dict_with_chapters = reviews['QZuisQ0Z6Sk']
product_review_dict_chapters = transcript_segmenter.get_product_review_dict(sample_review_dict_with_chapters, product_indicators, product_lexicon)

print("With chapters")
for k, v in product_review_dict_chapters.items():
  print(k)
  print(v)

Without chapters
makeup looks
{'original': 'Thank you guys, so much for watching let me know what other makeup looks you want me to do Bye'}
Becca Undereye Brightening Corrector
{'original': 'I m going in with the Becca Under  Eye Brightening Corrector and even though it s for eyes, I use it everywhere I need to correct discoloration then I m using the Too Faced Born This Way Concealer This stuff is very full coverage I wait 10 seconds before blending to give it time to set and we run it back And you guys know I have to go in twice with it to really cover up those dark circles for foundation  I m just applying dots to cover my face and I make sure to blend it out really well and, yes, always get that hairline and neck'}
Laura Mercier Setting Powder
{'original': 'I m setting my face with the Laura Mercier Setting Powder I go in with a sponge, first to set under my eyes, and then with a brush to lightly dust the rest of my face now I m using the rem'}
beauty Midnight Shadows Eyeshadow Pa

Loop to get all review data

In [ ]:
input_json_filename = "Feb23.json"
output_json_filename = "PRI_Output_"+input_json_filename
#Load data scraped using various Youtube API
with open(input_json_filename, 'r') as f:
  reviews = json.load(f)

In [ ]:
review_id_lst = []
for review_id, review_data in reviews.items():
  review_id_lst.append(review_id)
print(len(review_id_lst))
print(review_id_lst)

compiled_transcript_dict = {}
for review_id in review_id_lst:
  sample_review_dict = reviews[review_id]
  product_review_dict = transcript_segmenter.get_product_review_dict(sample_review_dict, product_indicators, product_lexicon)
  compiled_transcript_dict[review_id] = product_review_dict


In [ ]:
with open(output_json_filename, "w") as write_file:
   json.dump(compiled_transcript_dict, write_file, indent=4)